In [2]:
import pandas as pd 
import numpy as np
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from nltk.stem import PorterStemmer

# DATASET EXPLORING
<br>

## READ DATA

In [18]:
data = pd.read_csv("mail-data/mail-all.csv")

# TF-IDF
<br>

In [9]:
class CountVectorizer:
    def __init__(self):
        self.tokenizer = TweetTokenizer()
        self.stop_words = set(stopwords.words('french'))
        self.vocab = None
        self.ps = PorterStemmer()

    def preprocess_text(self, text):

        text = text.lower()
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        tokens = self.tokenizer.tokenize(text)
        tokens = [self.ps.stem(token) for token in tokens if token.isalpha() and token not in self.stop_words]
        
        return tokens

    def fit_transform(self, documents):
        tokenized_documents = [self.preprocess_text(doc) for doc in documents]
        # Création du vocabulaire et des fréquences des mots
        self.vocab = Counter()
        for doc in tokenized_documents:
            self.vocab.update(doc)

        # Transformation des documents en bag-of-words
        count_vectors = []
        for doc in tokenized_documents:
            count_vector = [doc.count(term) for term in self.vocab]
            count_vectors.append(count_vector)

        return count_vectors

    def transform(self, documents):
        tokenized_documents = [self.preprocess_text(doc) for doc in documents]
        
        # Transformation des documents en bag-of-words
        count_vectors = []
        for doc in tokenized_documents:
            count_vector = [doc.count(term) for term in self.vocab]
            count_vectors.append(count_vector)

        return count_vectors

    def get_feature_names_out(self):
        return list(self.vocab.keys())

In [11]:
import numpy as np

class TfidfVectorizer:
    def __init__(self, count_vectorizer):
        self.count_vectorizer = count_vectorizer
        self.vocab_size = None
        self.idf = None
        self.feature_names = None

    def fit_transform(self, documents):
        # Step 1: Calcul de la matrice term frequency
        count_vectors = self.count_vectorizer.fit_transform(documents)

        self.feature_names = self.count_vectorizer.get_feature_names_out()

        # Step 2: Calcul de l' inverse document frequency
        self.vocab_size = len(self.count_vectorizer.vocab)
        doc_freq = np.sum(np.array(count_vectors) > 0, axis=0)
        self.idf = np.log((len(documents) + 1) / (doc_freq + 1)) + 1

        # Step 3: Normalisation de la matrice term frequency avec l'inverse document frequency
        tfidf_matrix = np.array(count_vectors) * self.idf

        # Step 4: Normalisation de la matrice TF-IDF par la norme Euclidiènne
        norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
        norms[norms == 0] = 1e-12  # Remplacement des zéros
        tfidf_matrix /= norms

        return tfidf_matrix

    def transform(self, documents):
        # Step 1: Calcul de la matrice term frequency
        count_vectors = self.count_vectorizer.transform(documents)

        # Step 2: Normalisation de la matrice term frequency avec l'inverse document frequency
        tfidf_matrix = np.array(count_vectors) * self.idf

        # Step 3: Normalisation de la matrice TF-IDF par la norme Euclidiènne
        norms = np.linalg.norm(tfidf_matrix, axis=1, keepdims=True)
        norms[norms == 0] = 1e-12  # Replace zero norms with a small positive value
        tfidf_matrix /= norms

        return tfidf_matrix


In [12]:

text = data['texte']

# Create the  Count-vectorizer
count_vectorizer = CountVectorizer()

# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer(count_vectorizer)

# Fit the vectorizer on the text and transform the text into TF-IDF vectors
tfidf_vectors = vectorizer.fit_transform(text)

# Get the feature names (words) from the vectorizer
feature_names = vectorizer.feature_names

# Create a dataframe from the TF-IDF vectors
tfidf_dataframe = pd.DataFrame(tfidf_vectors, columns=feature_names)


In [13]:
tfidf_dataframe.insert(0, 'classe_of_mail', data['classe_mail'])

In [15]:
tfidf_dataframe.to_csv('mail-data/mail-tfidf.csv')